In [123]:
import numpy as np
import csv
from PIL import Image
import tensorflow as tf
import tensorlayer as tl
from tensorlayer.layers import *

In [119]:
action_list=[]
img_list=[]

In [120]:
file = open('roll_record.csv', 'r', encoding='utf-8')
saved_file = csv.reader(file)

action_list = []
for line in saved_file:
    action = line[0]
    action_list.append(action)
#print(action_list)
file.close()  

In [121]:
for i in range(0,212):
    input_img = Image.open("D:\Imitation-Learning-Dagger-Torcs\dcu\image\image_%d.jpg" % i)
    img_list.append(input_img)

In [122]:
def img_reshape(input_img):
    """ (3, 64, 64) --> (64, 64, 3) """
    _img = np.transpose(input_img, (1, 2, 0))
    _img = np.flipud(_img)
    _img = np.resize(_img, (1, 64, 64, 3))
    return _img

In [147]:
# Get roll input only now
num_actions = 1

In [148]:
images_all = np.zeros((0, 64, 64, 3))  # 처음 images_all 초기화
actions_all = np.zeros((0, num_actions)) # 0이 None일 수 있음 
#rewards_all = np.zeros((0,))

In [149]:
print("#"*50)  
print('Packing data into arrays... ')

##################################################
Packing data into arrays... 


In [151]:
print("#"*50)  
print('Packing data into arrays... ')  # Data패킹해주는 코드 아래에
for img, act in zip(img_list, action_list): # 받은 img_list가 이미 있고 그로부터 하나씩 변환해서 다시쌓기
    images_all = np.concatenate([images_all, img_reshape(img)], axis=0)  # images_all에 Data 패킹해주고
    actions_all = np.concatenate([actions_all, np.reshape(act, [1,num_actions])], axis=0) # action 도 패킹해주고
    #rewards_all = np.concatenate([rewards_all, rew], axis=0)

##################################################
Packing data into arrays... 


In [154]:
images_all.shape

(212, 64, 64, 3)

In [155]:
actions_all.shape

(212, 1)

In [158]:
# Save the expert's data
tl.files.save_any_to_npy(save_dict={'im': images_all, 'act': actions_all}, name='_tmp.npy')

# When loading the expert's data
# data = tl.files.load_npy_to_any(name='_tmp.npy')
# images_all = data['im']; actions_all = data['act']; rewards_all = data['re']

In [161]:
# save every 10th expert's observation to train 나중에 10으로 바꿔
# can check in teacher's folder
tl.files.exists_or_mkdir('image/teacher', verbose=True)
for i in range(0, len(images_all), 1):
    tl.vis.save_image(images_all[i], 'image/teacher/im_%d.png' % i)

[*] creates image/teacher ...


In [167]:
img_dim = [64, 64, 3]
n_action = 1        # steer only (float, left and right 1 ~ -1)
steps = 1000        # maximum step for a game
batch_size = 32
n_epoch = 100

In [169]:
###================= Define model
class Agent(object):
    def __init__(self, name='model', sess=None):
        assert sess != None
        self.name = name
        self.sess = sess

        self.x = tf.placeholder(tf.float32, [None, 64, 64, 3], name='Observaion')
        self.y = tf.placeholder(tf.float32, [None, 1], name='Steer')

        self._build_net(True, False)
        self._build_net(False, True)
        self._define_train_ops()

        tl.layers.initialize_global_variables(self.sess)

        print()
        self.n_test.print_layers()
        print()
        self.n_test.print_params(False)
        print()
        # exit()

    def _build_net(self, is_train=True, reuse=None):
        with tf.variable_scope(self.name, reuse=reuse) as vs:
            tl.layers.set_name_reuse(reuse)

            n = InputLayer(self.x / 255, name='in')

            n = Conv2d(n, 32, (3, 3), (1, 1), tf.nn.relu, "VALID", name='c1/1')
            n = Conv2d(n, 32, (3, 3), (1, 1), tf.nn.relu, "VALID", name='c1/2')
            n = MaxPool2d(n, (2, 2), (2, 2), 'VALID', name='max1')

            n = DropoutLayer(n, 0.75, is_fix=True, is_train=is_train, name='drop1')

            n = Conv2d(n, 64, (3, 3), (1, 1), tf.nn.relu, "VALID", name='c2/1')
            n = Conv2d(n, 64, (3, 3), (1, 1), tf.nn.relu, "VALID", name='c2/2')
            n = MaxPool2d(n, (2, 2), (2, 2), 'VALID', name='max2')
            # print(n.outputs)
            n = DropoutLayer(n, 0.75, is_fix=True, is_train=is_train, name='drop2')

            n = FlattenLayer(n, name='f')
            n = DenseLayer(n, 512, tf.nn.relu, name='dense1')
            n = DropoutLayer(n, 0.5, is_fix=True, is_train=is_train, name='drop3')
            n = DenseLayer(n, 1, tf.nn.tanh, name='o')

        if is_train:
            self.n_train = n
        else:
            self.n_test = n

    def _define_train_ops(self):
        self.cost = tl.cost.mean_squared_error(self.n_train.outputs, self.y, is_mean=False)
        self.train_params = tl.layers.get_variables_with_name(self.name, train_only=True, printable=False)
        self.train_op = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(self.cost, var_list=self.train_params)

    def train(self, X, y, n_epoch=100, batch_size=10, print_freq=20):
        for epoch in range(n_epoch):
            start_time = time.time()
            total_err, n_iter = 0, 0
            for X_, y_ in tl.iterate.minibatches(X, y, batch_size, shuffle=True):
                _, err = self.sess.run([self.train_op, self.cost], feed_dict={self.x: X_, self.y: y_})
                total_err += err
                n_iter += 1
            if epoch % print_freq == 0:
                print("Epoch [%d/%d] cost:%f took:%fs" % (epoch, n_epoch, total_err/n_iter, time.time()-start_time))

    def predict(self, image):
        a = self.sess.run(self.n_test.outputs, {self.x : image})
        return a

    def save_model(self):
        tl.files.save_npz(self.n_test.all_params, name=self.name+'.npz', sess=self.sess)

    def load_model(self):
        tl.files.load_and_assign_npz(sess=self.sess, name=self.name+'.npz', network=self.n_test)

###===================== Pretrain model using data for demonstration
sess = tf.InteractiveSession()
model = Agent(name='test1', sess=sess)
model.train(images_all, actions_all, n_epoch=n_epoch, batch_size=batch_size)
# save model after pretraining
model.save_model()
# model.load_model()
output_file = open('results.txt', 'w')

  [TL] InputLayer  test1/in: (?, 64, 64, 3)
  [TL] Conv2dLayer test1/c1/1: shape:[3, 3, 3, 32] strides:[1, 1, 1, 1] pad:VALID act:relu
  [TL] Conv2dLayer test1/c1/2: shape:[3, 3, 32, 32] strides:[1, 1, 1, 1] pad:VALID act:relu
  [TL] PoolLayer   test1/max1: ksize:[1, 2, 2, 1] strides:[1, 2, 2, 1] padding:VALID pool:max_pool
  [TL] DropoutLayer test1/drop1: keep:0.750000 is_fix:True
  [TL] Conv2dLayer test1/c2/1: shape:[3, 3, 32, 64] strides:[1, 1, 1, 1] pad:VALID act:relu
  [TL] Conv2dLayer test1/c2/2: shape:[3, 3, 64, 64] strides:[1, 1, 1, 1] pad:VALID act:relu
  [TL] PoolLayer   test1/max2: ksize:[1, 2, 2, 1] strides:[1, 2, 2, 1] padding:VALID pool:max_pool
  [TL] DropoutLayer test1/drop2: keep:0.750000 is_fix:True
  [TL] FlattenLayer test1/f: 10816
  [TL] DenseLayer  test1/dense1: 512 relu
  [TL] DropoutLayer test1/drop3: keep:0.500000 is_fix:True
  [TL] DenseLayer  test1/o: 1 tanh
  [TL] InputLayer  test1/in: (?, 64, 64, 3)
  [TL] Conv2dLayer test1/c1/1: shape:[3, 3, 3, 32] strides